In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [2]:
raw_df_match_details = pd.read_csv('../data/raw/match_details.csv')
raw_df_player_attr = pd.read_csv('../data/raw/player_attributes.csv')

In [3]:
raw_df_player_attr

,id,player_fifa_api_id,player_api_id,date,overall_rating,potential,preferred_foot,attacking_work_rate,defensive_work_rate,crossing,...,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
0,1,218353,505942,2016-02-18 00:00:00,67.0,71.0,right,medium,medium,49.0,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
1,2,218353,505942,2015-11-19 00:00:00,67.0,71.0,right,medium,medium,49.0,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
2,3,218353,505942,2015-09-21 00:00:00,62.0,66.0,right,medium,medium,49.0,...,54.0,48.0,65.0,66.0,69.0,6.0,11.0,10.0,8.0,8.0
3,4,218353,505942,2015-03-20 00:00:00,61.0,65.0,right,medium,medium,48.0,...,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0
4,5,218353,505942,2007-02-22 00:00:00,61.0,65.0,right,medium,medium,48.0,...,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183973,183974,102359,39902,2009-08-30 00:00:00,83.0,85.0,right,medium,low,84.0,...,88.0,83.0,22.0,31.0,30.0,9.0,20.0,84.0,20.0,20.0
183974,183975,102359,39902,2009-02-22 00:00:00,78.0,80.0,right,medium,low,74.0,...,88.0,70.0,32.0,31.0,30.0,9.0,20.0,73.0,20.0,20.0
183975,183976,102359,39902,2008-08-30 00:00:00,77.0,80.0,right,medium,low,74.0,...,88.0,70.0,32.0,31.0,30.0,9.0,20.0,73.0,20.0,20.0
183976,183977,102359,39902,2007-08-30 00:00:00,78.0,81.0,right,medium,low,74.0,...,88.0,53.0,28.0,32.0,30.0,9.0,20.0,73.0,20.0,20.0


In [4]:
# raw_df_match_details.info()

In [5]:
# raw_df_player_attr.info()

In [6]:
raw_df_match_details['date'] = pd.to_datetime(raw_df_match_details['date'])
raw_df_player_attr['date'] = pd.to_datetime(raw_df_player_attr['date'])
raw_df_match_details.sort_values(by=['season', 'date'], inplace=True)

In [7]:
def get_points(row, team):
    points_mapping = {
        'H': {'home': 3, 'away': 0},
        'D': {'home': 1, 'away': 1},
        'A': {'home': 0, 'away': 3}
    }
    team_type = 'home' if row['home_team'] == team else 'away'
    return int(points_mapping[row['result_match']][team_type])

def process_points(team, df, match_date, match_season):
    team_matches = df.query('(home_team == @team | away_team == @team) & (season == @match_season & date < @match_date)')
    if len(team_matches) == 0:
        return 0

    return team_matches.apply(lambda row: get_points(row, team), axis=1).sum()

def count_points(match_row, df):
    match_date = match_row['date']
    match_season = match_row['season']
    home_team = match_row['home_team']
    away_team = match_row['away_team']

    home_team_points = process_points(home_team, df, match_date, match_season)
    away_team_points = process_points(away_team, df, match_date, match_season)

    return home_team_points, away_team_points

raw_df_match_details[['points_home', 'points_away']] = raw_df_match_details.apply(lambda row: count_points(row, raw_df_match_details), axis=1, result_type='expand')

In [8]:
raw_df_match_details['result_match']

0       H
1       A
2       H
3       A
4       H
       ..
3035    H
3036    D
3037    D
3038    D
3039    H
Name: result_match, Length: 3040, dtype: object

In [9]:
raw_df_match_details = (raw_df_match_details
                        .assign(date=pd.to_datetime(raw_df_match_details['date']),
                                result_match=lambda df: (df['result_match'] == 'H').astype(int))
                        )

In [10]:
from playerstats import player_stats

players_cols = ['{}_player_{}'.format(team, i) for team in ['home', 'away'] for i in range(1, 12)]

player_stats_dict_series = raw_df_match_details.apply(
    lambda row: player_stats.get_player_stat(
        match_row=row,
        df_matches=raw_df_match_details,
        df_player_attr=raw_df_player_attr,
        players=players_cols
    ),
    axis=1
)

new_player_stats_df = pd.json_normalize(player_stats_dict_series)

df = pd.merge(raw_df_match_details, new_player_stats_df, how='left', on='match_api_id')
df.drop(players_cols, axis=1, inplace=True)
matching_columns = [col for col in df.columns if 'player_rating' in col] + [col for col in df.columns if
                                                                            'aggression_rating' in col] + [col for col
                                                                                                           in df.columns
                                                                                                           if
                                                                                                           'strength_rating' in col] + [
                       col for col in df.columns if 'acceleration_rating' in col]
df.dropna(subset=matching_columns, inplace=True)
matching_columns_filtered = [c for c in df.columns if c != "date" and c != "season"]

In [11]:
from src.shiftdata.shift_data import ShiftDataPreprocessor

preprocessor = ShiftDataPreprocessor(df)

home_df = preprocessor.select_and_rename_columns('home_')
away_df = preprocessor.select_and_rename_columns('away_')

team_df = preprocessor.concatenate_teams(home_df, away_df)

features_to_shift = ['team_goal', 'team_shoton', 'team_possession']
team_df_shifted = preprocessor.shift_features(features_to_shift)

df_final = preprocessor.merge_shifted_features(team_df_shifted)

INFO: Merging shifted features back into the original DataFrame.


In [12]:
df_ = df_final.copy()
df_.sort_values(['date'], inplace=True)

In [13]:
home_strength_rating_cols = df_.filter(like='strength_rating_home').columns.to_list()
away_strength_rating_cols = df_.filter(like='strength_rating_away').columns.to_list()

home_aggression_rating_cols = df_.filter(like='aggression_rating_home').columns.to_list()
away_aggression_rating_cols = df_.filter(like='aggression_rating_away').columns.to_list()

home_overall_rating_cols = df_.filter(like='overall_rating_home').columns.to_list()
away_overall_rating_cols = df_.filter(like='overall_rating_away').columns.to_list()

home_acceleration_rating_cols = df_.filter(like='acceleration_rating_home').columns.to_list()
away_acceleration_rating_cols = df_.filter(like='acceleration_rating_away').columns.to_list()

home_possession_rating_cols = df_.filter(like='home_last_team_possession').columns.to_list()
away_possession_rating_cols = df_.filter(like='away_last_team_possession').columns.to_list()

home_shoton_rating_cols = df_.filter(like='home_last_team_shoton').columns.to_list()
away_shoton_rating_cols = df_.filter(like='away_last_team_shoton').columns.to_list()

In [14]:
from helper import rolling_avg

df_['team_strength_home'] = df_[home_strength_rating_cols].mean(axis=1)
df_['team_strength_away'] = df_[away_strength_rating_cols].mean(axis=1)
df_['strength_difference'] = df_['team_strength_home'] - df_['team_strength_away']

df_['team_aggression_home'] = df_[home_aggression_rating_cols].mean(axis=1)
df_['team_aggression_away'] = df_[away_aggression_rating_cols].mean(axis=1)
df_['aggression_difference'] = df_['team_aggression_home'] - df_['team_aggression_away']

df_['team_acceleration_home'] = df_[home_acceleration_rating_cols].mean(axis=1)
df_['team_acceleration_away'] = df_[away_acceleration_rating_cols].mean(axis=1)
df_['acceleration_difference'] = df_['team_acceleration_home'] - df_['team_acceleration_away']

df_['points_difference'] = df_['points_home'] - df_['points_away']

df_['goal_conversion_rate_home'] = df_['home_last_team_goal'] / (df_['home_last_team_shoton'] + 1e-6)
df_['goal_conversion_rate_away'] = df_['away_last_team_goal'] / (df_['away_last_team_shoton'] + 1e-6)

df_ = rolling_avg.calculate_rolling_avg_pandas(df_, 10)

columns_to_drop = (
    home_strength_rating_cols
    + away_strength_rating_cols
    + home_aggression_rating_cols
    + away_aggression_rating_cols
    + home_overall_rating_cols
    + away_overall_rating_cols
    + home_acceleration_rating_cols
    + away_acceleration_rating_cols
)

# Drop the valid columns
df_ = df_.drop(columns_to_drop, axis=1)

In [15]:
df_['possession_strength_ratio_home'] = (df_['home_last_team_possession'] / (df_['team_strength_home'] + 1e-6))
df_['possession_strength_ratio_away'] = (df_['away_last_team_possession'] / (df_['team_strength_away'] + 1e-6))

df_['rolling_avg_goals_ratio_away'] = (df_['rolling_avg_goals_home'] / (df_['rolling_avg_goals_away'] + 1e-6))


In [16]:
df_.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2975 entries, 0 to 2974
Data columns (total 34 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   match_api_id                    2975 non-null   int64         
 1   season                          2975 non-null   object        
 2   stage                           2975 non-null   int64         
 3   date                            2975 non-null   datetime64[ns]
 4   away_team                       2975 non-null   int64         
 5   home_team                       2975 non-null   int64         
 6   result_match                    2975 non-null   int64         
 7   points_home                     2975 non-null   int64         
 8   points_away                     2975 non-null   int64         
 9   home_last_team_goal             2975 non-null   float64       
 10  home_last_team_shoton           2975 non-null   float64       
 11  home

In [17]:
df_[['rolling_goal_stability_home', 'rolling_goal_stability_away']]

,rolling_goal_stability_home,rolling_goal_stability_away
0,0.000000,0.000000
1,0.000000,0.000000
2,0.000000,0.000000
3,0.000000,0.000000
4,0.000000,0.000000
...,...,...
2970,0.564767,0.232673
2971,0.175936,0.235879
2972,0.253970,0.505636
2973,0.380279,0.189439


In [18]:
df_.drop(columns=["match_api_id", "result_match", "season", "stage", "date", "home_team", "away_team"]).describe().round(2)

,points_home,points_away,home_last_team_goal,home_last_team_shoton,home_last_team_possession,away_last_team_goal,away_last_team_shoton,away_last_team_possession,team_strength_home,team_strength_away,...,points_difference,goal_conversion_rate_home,goal_conversion_rate_away,rolling_avg_goals_home,rolling_goal_stability_home,rolling_avg_goals_away,rolling_goal_stability_away,possession_strength_ratio_home,possession_strength_ratio_away,rolling_avg_goals_ratio_away
count,2975.00,2975.00,2975.00,2975.00,2975.00,2975.00,2975.00,2975.00,2975.00,2975.00,...,2975.00,2975.00,2975.00,2975.00,2975.00,2975.00,2975.00,2975.00,2975.00,2975.00
mean,25.65,25.89,1.81,6.20,48.84,1.93,5.90,51.05,68.51,68.78,...,-0.23,0.41,0.46,1.86,0.34,1.87,0.34,0.72,0.75,1.06
std,17.78,17.70,0.92,3.32,8.71,1.02,3.18,8.75,6.64,6.52,...,13.87,0.39,0.44,0.47,0.23,0.47,0.24,0.15,0.15,0.38
min,0.00,0.00,1.00,1.00,20.00,1.00,1.00,4.00,45.02,46.90,...,-54.00,0.05,0.05,1.00,0.00,1.00,0.00,0.27,0.05,0.29
25%,12.00,12.00,1.00,4.00,43.00,1.00,4.00,45.00,64.47,64.66,...,-7.00,0.19,0.20,1.53,0.18,1.55,0.18,0.62,0.65,0.79
50%,23.00,23.00,1.67,6.00,49.00,2.00,5.00,51.00,68.89,69.22,...,0.00,0.29,0.33,1.80,0.27,1.81,0.27,0.71,0.75,1.00
75%,36.00,37.00,2.00,8.00,54.00,2.00,8.00,56.50,72.83,73.12,...,6.00,0.50,0.50,2.12,0.42,2.12,0.42,0.81,0.84,1.26
max,86.00,88.00,8.00,24.00,79.00,9.00,26.00,80.00,85.41,84.61,...,51.00,5.00,8.00,4.28,2.44,4.38,2.44,1.27,1.35,3.45


In [19]:
df_.to_csv('../data/preprocessed/preprocessed_1.csv', index=False)

In [20]:
df_.filter(like='goal_conversion_rate').describe().round(2)

,goal_conversion_rate_home,goal_conversion_rate_away
count,2975.00,2975.00
mean,0.41,0.46
std,0.39,0.44
min,0.05,0.05
25%,0.19,0.20
50%,0.29,0.33
75%,0.50,0.50
max,5.00,8.00
